## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Assomada,CV,15.1000,-23.6833,69.35,83,5,13.60,clear sky
1,1,Punta Arenas,CL,-53.1500,-70.9167,46.51,57,0,13.80,clear sky
2,2,Hobart,AU,-42.8794,147.3294,51.66,68,40,9.22,scattered clouds
3,3,Chuy,UY,-33.6971,-53.4616,58.60,89,100,2.73,overcast clouds
4,4,Tuktoyaktuk,CA,69.4541,-133.0374,32.41,74,36,8.57,scattered clouds
5,5,Petropavlovsk-Kamchatskiy,RU,53.0452,158.6483,57.79,44,75,13.42,broken clouds
6,6,Peniche,PT,39.3558,-9.3811,62.29,85,43,10.80,scattered clouds
7,7,Juneau,US,58.3019,-134.4197,69.84,72,75,10.36,broken clouds
8,8,Avarua,CK,-21.2078,-159.7750,82.45,78,17,11.50,few clouds
9,9,Vila Velha,BR,-20.3297,-40.2925,67.95,77,0,3.44,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp =float(input("What is the minimum temperature you would like for your trip? "))

max_temp =float(input("What is the maximum temperature you would like for your trip? "))



What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preffered_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preffered_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Avarua,CK,-21.2078,-159.7750,82.45,78,17,11.50,few clouds
13,13,Yellowknife,CA,62.4560,-114.3525,75.56,25,80,17.00,broken clouds
16,16,Mayo,US,38.8876,-76.5119,78.19,49,0,8.05,clear sky
20,20,Bathsheba,BB,13.2167,-59.5167,79.41,94,75,20.71,broken clouds
24,24,Hilo,US,19.7297,-155.0900,89.28,80,75,9.22,broken clouds
28,28,Victoria,HK,22.2855,114.1577,86.18,87,97,8.99,overcast clouds
29,29,Chateaubelair,VC,13.2833,-61.2500,79.32,83,40,12.66,scattered clouds
35,35,Turayf,SA,31.6725,38.6637,77.25,27,0,11.50,clear sky
47,47,Kipini,KE,-2.5257,40.5262,76.39,85,73,21.41,light rain
50,50,Kapaa,US,22.0752,-159.3190,82.47,70,20,11.50,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
print(f'{preffered_df.isnull().sum()} \n \n')

print(preffered_df.count())


City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64 
 

City_ID                219
City                   219
Country                219
Lat                    219
Lng                    219
Max Temp               219
Humidity               219
Cloudiness             219
Wind Speed             219
Current Description    219
dtype: int64


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#there are no empty rows
clean_df = preffered_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Avarua,CK,82.45,few clouds,-21.2078,-159.7750,
13,Yellowknife,CA,75.56,broken clouds,62.4560,-114.3525,
16,Mayo,US,78.19,clear sky,38.8876,-76.5119,
20,Bathsheba,BB,79.41,broken clouds,13.2167,-59.5167,
24,Hilo,US,89.28,broken clouds,19.7297,-155.0900,
28,Victoria,HK,86.18,overcast clouds,22.2855,114.1577,
29,Chateaubelair,VC,79.32,scattered clouds,13.2833,-61.2500,
35,Turayf,SA,77.25,clear sky,31.6725,38.6637,
47,Kipini,KE,76.39,light rain,-2.5257,40.5262,
50,Kapaa,US,82.47,few clouds,22.0752,-159.3190,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
      #use variable row to take value of lat for current index in loop
    lat = row["Lat"]
    
    lng = row["Lng"]
    
     # Add the latitude and longitude to location key for the params dictionary.
        #create a new key value pair and add to dictionary
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search.
    
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        
        print("Hotel not found... skipping.")
    


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.tail()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
694,Olinda,BR,76.73,few clouds,-8.0089,-34.8553,Flat Quatro Rodas Olinda
695,Howard Springs,AU,82.87,scattered clouds,-12.4970,131.0470,buffalo beach hostel
697,Ariyalur,IN,78.84,overcast clouds,11.1333,79.0833,BERGAMONT HOTEL
698,Wang Saphung,TH,75.22,few clouds,17.3010,101.7685,โรงแรมวังรีสอร์ท
699,Mogadishu,SO,78.55,broken clouds,2.0371,45.3438,Hotel Juba


In [10]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df["Hotel Name"] = hotel_df["Hotel Name"].str.strip().replace('',np.nan)
    
clean_hotel_df = hotel_df

clean_hotel_df.tail()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
694,Olinda,BR,76.73,few clouds,-8.0089,-34.8553,Flat Quatro Rodas Olinda
695,Howard Springs,AU,82.87,scattered clouds,-12.4970,131.0470,buffalo beach hostel
697,Ariyalur,IN,78.84,overcast clouds,11.1333,79.0833,BERGAMONT HOTEL
698,Wang Saphung,TH,75.22,few clouds,17.3010,101.7685,โรงแรมวังรีสอร์ท
699,Mogadishu,SO,78.55,broken clouds,2.0371,45.3438,Hotel Juba


In [12]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"


# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>

<dt>Hotel Name</dt><dd>{Hotel Name}</dd>

<dt>City Name</dt><dd>{City}</dd>

<dt>Country</dt><dd>{Country}</dd>

<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>

</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Call the figure to plot the data.

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))